In [5]:
import sys, os
sys.path.append('..')

import datetime as dt
from capture_dou.inlabs_driver import InLabsDriver

#tipo_dou="DO1 DO2 DO3 DO1E DO2E DO3E" # Seções separadas por espaço
tipo_dou = "DO2"

In [3]:
def brasilia_day():
    """
    No matter where the code is ran, return UTC-3 day
    (Brasilia local day, no daylight savings)
    """
    return (dt.datetime.utcnow() + dt.timedelta(hours=-3)).replace(hour=0, minute=0, second=0, microsecond=0)

In [8]:
import requests
import io
import zipfile

def parse_zipped_response(response):
    """
    Download a ZIP file and extract its contents in memory
    yields (filename, file-like object) pairs
    """
    with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
        for zipinfo in thezip.infolist():
            with thezip.open(zipinfo) as thefile:
                yield zipinfo.filename, thefile

In [37]:
with zipfile.ZipFile(io.BytesIO(file_response.content)) as thezip:
    for zipinfo in thezip.infolist()[:1]:
        print(zipinfo.filename)
        with thezip.open(zipinfo) as thefile:
            print(thefile.read(5000))

529_20230112_20228383.xml
b'\xef\xbb\xbf<xml><article id="30058734" name="Po 01 Aposentadoria FRANCISCO MA" idOficio="9331015" pubName="DO2" artType="Portaria" pubDate="12/01/2023" artClass="00011:00008:00000:00000:00000:00000:00000:00000:00000:00000:00017:00000" artCategory="Minist\xc3\xa9rio da Ci\xc3\xaancia, Tecnologia e Inova\xc3\xa7\xc3\xb5es/Centro de Tecnologia Mineral" artSize="12" artNotes="" numberPage="4" pdfPage="http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=12/01/2023&amp;jornal=529&amp;pagina=4" editionNumber="9" highlightType="" highlightPriority="" highlight="" highlightimage="" highlightimagename="" idMateria="20228383">\r\n  <body>\r\n    <Identifica><![CDATA[PORTARIA CETEM N\xc2\xba 1, DE 3 DE JANEIRO DE  2023]]></Identifica>\r\n    <Data><![CDATA[]]></Data>\r\n    <Ementa />\r\n    <Titulo />\r\n    <SubTitulo />\r\n    <Texto><![CDATA[<p class="identifica">PORTARIA CETEM N\xc2\xba 1, DE 3 DE JANEIRO DE 2023</p><p>A Diretora do Centro de Tecnologi

In [14]:

# Inicialização do driver:
driver = InLabsDriver()
driver.login()

# Montagem da URL:
do_date_format = '%Y-%m-%d'
# Transforms date to DOU format:
date_string    = brasilia_day().strftime(do_date_format)

for dou_secao in tipo_dou.split(' '):
    file_url = driver.url_download + date_string + "&dl=" + date_string + "-" + dou_secao + ".zip"
    file_header = {'Cookie': 'inlabs_session_cookie=' + driver.cookie, 'origem': '736372697074'}
    file_response = driver.session.request("GET", file_url, headers = file_header)
    if file_response.status_code == 200:
        data = parse_zipped_response(file_response)

/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'inlabs.in.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'inlabs.in.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'inlabs.in.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.w

In [15]:
lista = list(data)

In [36]:
lista[0][1].read(100)

ValueError: read from closed file.

In [30]:
with zipfile.ZipFile(lista[0][0], 'r') as zip:
    zip.open()

FileNotFoundError: [Errno 2] No such file or directory: '529_20230112_20228383.xml'